# Discovering faiss

This is a test notebook that I created to discover, understand and play with faiss parameters. It uses 400k lines of randomly-selected data from the "Grand-Débat" project.

In [1]:
import faiss
import numpy as np

In [2]:
dimension = 128
dataset_size = 100_000
np.random.seed(2022)

dataset = np.random.random((dataset_size, dimension)).astype('float32')

## Setting up a flat index

A flat index contains all of the data and uses an exhaustive search. It is neither memory- nor speed- efficient, but it does the job when the dataset isn't too big. It is also the only index that will return exact results every time. 

In [3]:
index_flat = faiss.IndexFlatL2(dimension)
index_flat.add(dataset)

We search for the first three vectors in the dataset, and we ask faiss for the five closest vectors to our query. `index.search()` returns two arrays of size (3, 5) : the array of distances and the array of indices. 

In [41]:
# %%timeit
distances, indices = index_flat.search(dataset[:3], k=5)

The first row in the array of distances contains the distances from our search results to the first vector in our query. The first distance is 0 (the closest vector to our query is itself), and the next 5 distances are increasing as we are getting farther from our query. Same thing for the next two rows.

The array of indices shows the indices of our search results. In each row, the first entry (the index of the vector closest to our query) is the index of the vector itself. All good.

In [42]:
print(distances)
print([distance.mean() for distance in distances])
print("\n")
print(indices)

[[ 0.       14.362421 14.602067 14.767819 15.029004]
 [ 0.       12.729409 13.443033 13.679935 14.520304]
 [ 0.       13.01179  13.469017 13.666645 13.668671]]
[11.752262, 10.8745365, 10.763225]


[[    0 18096 23340 69719 76760]
 [    1 21442 14241 16511 88648]
 [    2 31767 77193 90699 42060]]


## Speeding up search with Inverse File Indexing (IVF)

IVF separates the vectors into clusters (cells); at search time, only those cells whose centroids are close enough to the query are visited. The number of cells that are created, and the number of cells that are visited when searching are user-determined hyperparameters. More cells mean fewer vectors per cell, and a faster search; but it also means the results might be inaccurate (for example, when a vector falls close to the boundary between two cells).

### Playing with `ncells` and `nprobe`

Only one cell corresponds to a flat index. We'll try with 10 cells, which means that each cell has roughly 10000 vectors in it

In [19]:
n_cells = 10

quantizer = faiss.IndexFlatL2(dimension)
index_ivf_10 = faiss.IndexIVFFlat(quantizer, dimension, n_cells)

In [20]:
# %%timeit

index_ivf_10.train(dataset)
index_ivf_10.add(dataset)

`nprobe = 1` means we're only looking at one cell. Unsurprisingly, the search is about 10 times faster (since we're only visiting a tenth of the data).

In [36]:
# %%timeit
index_ivf_10.nprobe = 1
distances, indices = index_ivf_10.search(dataset[:3], k=5)

We can see that the distances are on average a bit higher than the ones obtained with the flat index. This illustrates the aforementioned accuracy-speed tradeoff. If we set `nprobe` higher, we'll get better results, but we'll be comparing our query to more data points, which will take longer.

In [37]:
print(distances)
print([distance.mean() for distance in distances])
print("\n")
print(indices)

[[ 0.       16.018751 16.067871 16.07457  16.079603]
 [ 0.       12.729409 13.443033 15.023331 15.109427]
 [ 0.       13.793472 14.110714 14.26668  14.326261]]
[12.84816, 11.261041, 11.299425]


[[    0 50348 10005 16734 19739]
 [    1 21442 14241 66051 88838]
 [    2 79805 48259 41983 42966]]


Let's go to the other end of the spectrum with 1 000 cells (so only about 100 vectors per cell).

In [27]:
n_cells = 1000

quantizer = faiss.IndexFlatL2(dimension)
index_ivf_1000 = faiss.IndexIVFFlat(quantizer, dimension, n_cells)

index_ivf_1000.train(dataset)
index_ivf_1000.add(dataset)

Search is even faster (but not 100 times faster), but the results are less accurate.

In [38]:
# %%timeit 
index_ivf_1000.nprobe = 1
distances, indices = index_ivf_1000.search(dataset[:3], k=5)

In [40]:
print(distances)
print([distance.mean() for distance in distances])
print("\n")
print(indices)

[[ 0.       15.554035 16.189127 17.478779 17.842056]
 [ 0.       15.568712 15.874627 16.26919  16.367075]
 [ 0.       13.848631 14.743672 14.788898 15.357994]]
[13.412799, 12.815921, 11.747839]


[[    0  6183 59460 61640 87416]
 [    1 26349 59134 44111 81169]
 [    2 94523 61611 79798 89559]]


Since there is no data compression, the resulting index is 52Mo on disk. That's exactly the size of the whole dataset, unsurprisingly.

```python
    faiss.write_index(index_ivf_10, "index_ivf_10.faiss")
```

## Reducing index size with product quantization

Product quantization reduces the size of vectors via compression.

The gist of it:
- First, all the vectors are clustered into $n$ cells. Each cell has a centroid.
- Then, each vector is cut up into $m$ segments of dimension $d/m$ (where $d$ is the original dimension of the embedding). Each segment of a vector is compared to the corresponding segment of each of the centroids. We store the number of the centroid segment to which it is closest.

For example, imagine our vectors have dimension $d=100$. We cluster them into $n=5$ cells, and divide them into $2$ segments (of dimension $50$ each). Pick a vector $v$. If the first segment of $v$ is closest to the first segment of the 5th centroid, the second segment is closest to the second segment of the 2nd centroid, then we store the coordinates of $v$ as $(5, 2)$.

When comparing a new vector $w$ to all the vectors in the database, we just have to compute the distances between $w$ and each of the centroid segments.

Let's try this with 100 centroids and 8 segments.

In [44]:
n_cells = 100 # number of Voronoi cells/centroids
m = 8 # number of segments    
n_bits = 8 # number of bits on which to encode each subvector                 
quantizer = faiss.IndexFlatL2(dimension) 
index_ivfpq_100 = faiss.IndexIVFPQ(quantizer, dimension, n_cells, m, n_bits)

index_ivfpq_100.train(dataset)
index_ivfpq_100.add(dataset)

Since there are more steps involved, there is a bit of training overhead.

We start with `nprobe = 1`.

In [46]:
# %%timeit
index_ivfpq_100.nprobe = 1
distances, indices = index_ivfpq_100.search(dataset[:3], k=5)

Interestingly, even though the first result in each row is our original query, note that none of the distances are zero: this is because of lossy compression.

In [47]:
print(distances)
print([distance.mean() for distance in distances])
print("\n")
print(indices)

[[ 5.5896006 12.378974  12.777773  12.822787  12.896865 ]
 [ 6.126076  10.91538   11.37427   11.887591  12.234667 ]
 [ 5.219045  10.93168   11.196173  11.346197  11.445152 ]]
[11.2932, 10.507597, 10.027649]


[[    0 66299 19557 16486 64001]
 [    1  5164 75208 61405 94094]
 [    2 89565 94858 47596 82196]]


Even when we probe all the cells, the results aren't those returned by the flat index; again, this is due to lossy compression.

In [49]:
# %%timeit
index_ivfpq_100.nprobe = 100
distances, indices = index_ivfpq_100.search(dataset[:3], k=5)

print(distances)
print([distance.mean() for distance in distances])
print("\n")
print(indices)

[[ 5.5896006 11.687067  11.887678  11.897254  12.100771 ]
 [ 6.126076  10.91538   11.355385  11.37427   11.771835 ]
 [ 5.219045  10.795271  10.859158  10.887173  10.93168  ]]
[10.632474, 10.308589, 9.738465]


[[    0 23340 17245 46019 98771]
 [    1  5164 90856 75208 46210]
 [    2 48584 83930 31179 89565]]


The index on disk is small: a bit more than 1Mo, for an original dataset size of 52Mo.

```python
    faiss.write_index(index_ivfpq_100, "index_ivfpq_100_test.faiss")
```

OK, now let's try with more segments. Training time is going to be longer.

In [51]:
n_cells = 100 # number of Voronoi cells/centroids
m = 16 # number of segments    
n_bits = 8 # number of bits on which to encode each subvector                 
quantizer = faiss.IndexFlatL2(dimension) 
index_ivfpq_100_16 = faiss.IndexIVFPQ(quantizer, dimension, n_cells, m, n_bits)

index_ivfpq_100_16.train(dataset)
index_ivfpq_100_16.add(dataset)

More segments should lead to more accurate results, although it is hard to measure here; the distance of the query to itself is definitely lower, which means that our compression is less lossy than previously; the distance to the next nearest vectors is higher: this could simply be a result of more accurate compression (the vectors are more "separated").

In [57]:
# %%timeit
index_ivfpq_100_16.nprobe = 1
distances, indices = index_ivfpq_100_16.search(dataset[:3], k=5)

print(distances)
print([distance.mean() for distance in distances])
print("\n")
print(indices)

[[10.401583  11.299492  11.376704  11.45536   11.519587 ]
 [10.267775  10.756173  10.909281  11.085212  11.124319 ]
 [ 7.9992633  8.781995   9.142373   9.184871   9.217779 ]]
[11.210546, 10.828552, 8.865255]


[[    0 48054 59030  8686 10504]
 [    1 16914  7473 56099 93304]
 [    2 61116 94983 94538 13337]]


Drastically reducing the number of cells and segments can lead to inexact results: two of the results of our search (the second and third closest vectors for query number 3) have the exact same distance to the query. 

With more data and a different distribution, we could get to a point where several points share the same distance to the query as itself; the results would then be a random selection of these points.

In [68]:
n_cells = 8 # number of Voronoi cells/centroids
m = 2 # number of segments    
n_bits = 8 # number of bits on which to encode each subvector                 
quantizer = faiss.IndexFlatL2(dimension) 
index_ivfpq_100_2 = faiss.IndexIVFPQ(quantizer, dimension, n_cells, m, n_bits)

index_ivfpq_100_2.train(dataset)
index_ivfpq_100_2.add(dataset)

In [69]:
# %%timeit
index_ivfpq_100_2.nprobe = 1
distances, indices = index_ivfpq_100_2.search(dataset[:3], k=5)

print(distances)
print([distance.mean() for distance in distances])
print("\n")
print(indices)

[[10.460514  10.775399  10.976932  11.01385   11.029998 ]
 [10.283417  10.727434  10.741786  10.8827505 10.905673 ]
 [ 8.392833   8.899557   8.899557   9.009148   9.168215 ]]
[10.851339, 10.708212, 8.873861]


[[    0 27592 39701 68029 23513]
 [    1 11372 55097 75407  4630]
 [    2 47887 78694 70611 65424]]


## Battling the curse of dimensionality with Hierarchical Navigable Small Worlds

We first look at a flat HNSW index; it will store the full dataset in memory, plus some additional data for connections. HNSW is *not* a memory-efficient index, and for a small dataset like ours, where we get pretty good performance with a flat index, it isn't actually worth it.

Besides the dimension, the index requires a number of connections per node. More connections means a faster search, but also more memory (and training) overhead.

In [81]:
connections = 32 # number of connections per node    

index_hnsw = faiss.IndexHNSWFlat(dimension, connections)

index_hnsw.train(dataset)
index_hnsw.add(dataset)

In [82]:
# %%timeit
distances, indices = index_hnsw.search(dataset[:3], k=5)

print(distances)
print([distance.mean() for distance in distances])
print("\n")
print(indices)

[[ 0.        14.767819  15.123554  15.206737  15.414215 ]
 [ 0.        13.443033  13.679935  14.520304  14.807053 ]
 [ 0.        13.469017  14.0393095 14.462894  14.582859 ]]
[12.102465, 11.290065, 11.310816]


[[    0 69719 17614 47744 91427]
 [    1 14241 16511 88648 71178]
 [    2 77193 31179  6502 91193]]


Decreasing the number of connections decreases training time and size in memory; it also decreases precision: none of our query vectors has been returned by the search.

In [79]:
connections = 2 # number of connections per node    

index_hnsw_8 = faiss.IndexHNSWFlat(dimension, connections)

index_hnsw_8.train(dataset)
index_hnsw_8.add(dataset)

In [80]:
# %%timeit
distances, indices = index_hnsw_8.search(dataset[:3], k=5)

print(distances)
print([distance.mean() for distance in distances])
print("\n")
print(indices)

[[15.206737 16.417955 16.771797 17.316042 17.478779]
 [15.623398 15.993703 17.140484 17.235678 17.615704]
 [16.135687 16.454727 17.348625 17.541634 17.602398]]
[16.638264, 16.721792, 17.016613]


[[47744 14059 62028 80728 61640]
 [93616 75403 84150  8022 82434]
 [22449 46881  4132 39804 51984]]


On disk, the index is actually a little larger than the dataset itself, at 54 Mo.

```python
    faiss.write_index(index_hnsw_8, "../data/clean/indexes/index_hnsw_8_test.faiss")
````